In [1]:
!pip install polars numpy matplotlib scipy pingouin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 755.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 36.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 31.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 44.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.2 MB/s eta 0:00:00a 0:00:01


In [10]:
import pandas as pd
import pingouin as pg
from pathlib import Path
from scipy.stats import shapiro, ttest_rel, wilcoxon

# Data Preprocessing

In [11]:
outputs_path = Path('outputs')
csv_file = list(outputs_path.glob('*.csv'))
csv_file = sorted(csv_file)[0]

In [12]:
df = pd.read_csv(csv_file)

In [13]:
df

,approach,generation_model,evaluation_model,from_ending_type,classified_as,num_stories
0,compressed,claude-3-opus-20240229,claude-3-opus-20240229,positive,positive,100
1,compressed,claude-3-opus-20240229,claude-3-opus-20240229,positive,negative,0
2,compressed,claude-3-opus-20240229,claude-3-opus-20240229,positive,neutral,0
3,compressed,claude-3-opus-20240229,claude-3-opus-20240229,negative,positive,11
4,compressed,claude-3-opus-20240229,claude-3-opus-20240229,negative,negative,78
...,...,...,...,...,...,...
175,compressed,chat-bison-001,claude-3-opus-20240229,positive,bittersweet,0
176,compressed,chat-bison-001,claude-3-opus-20240229,neutral,redemptive,0
177,compressed,chat-bison-001,claude-3-opus-20240229,neutral,bittersweet,0
178,compressed,chat-bison-001,claude-3-opus-20240229,negative,redemptive,0


## Pre-process data to answer the following questions:
1. Does prompt compression make the model generate last chapters with ending of the same type as the given synopsis?

In [14]:
# filter only positive-positive, negaitve-negative, and neutral-neutral pairs
paired_df = df[df['from_ending_type'] == df['classified_as']]

In [15]:
paired_df

,approach,generation_model,evaluation_model,from_ending_type,classified_as,num_stories
0,compressed,claude-3-opus-20240229,claude-3-opus-20240229,positive,positive,100
4,compressed,claude-3-opus-20240229,claude-3-opus-20240229,negative,negative,78
8,compressed,claude-3-opus-20240229,claude-3-opus-20240229,neutral,neutral,37
10,compressed,gemini-1.0-pro,claude-3-opus-20240229,positive,positive,100
14,compressed,gemini-1.0-pro,claude-3-opus-20240229,negative,negative,79
18,compressed,gemini-1.0-pro,claude-3-opus-20240229,neutral,neutral,39
19,compressed,gpt-4-0125-preview,claude-3-opus-20240229,positive,positive,100
23,compressed,gpt-4-0125-preview,claude-3-opus-20240229,negative,negative,26
28,compressed,gpt-4-0125-preview,claude-3-opus-20240229,neutral,neutral,23
30,compressed,claude-2.1,claude-3-opus-20240229,positive,positive,100


In [21]:
baseline_paired_df = paired_df[paired_df['approach'] == 'baseline']
compressed_paired_df = paired_df[paired_df['approach'] == 'compressed']

In [22]:
baseline_paired_df

,approach,generation_model,evaluation_model,from_ending_type,classified_as,num_stories
58,baseline,claude-3-opus-20240229,claude-3-opus-20240229,positive,positive,100
62,baseline,claude-3-opus-20240229,claude-3-opus-20240229,negative,negative,73
67,baseline,claude-3-opus-20240229,claude-3-opus-20240229,neutral,neutral,30
69,baseline,gemini-1.0-pro,claude-3-opus-20240229,positive,positive,100
73,baseline,gemini-1.0-pro,claude-3-opus-20240229,negative,negative,88
78,baseline,gemini-1.0-pro,claude-3-opus-20240229,neutral,neutral,37
80,baseline,gpt-4-0125-preview,claude-3-opus-20240229,positive,positive,100
84,baseline,gpt-4-0125-preview,claude-3-opus-20240229,negative,negative,35
89,baseline,gpt-4-0125-preview,claude-3-opus-20240229,neutral,neutral,17
91,baseline,claude-2.1,claude-3-opus-20240229,positive,positive,100


# Normality Test

## Baseline

In [25]:
THRESHOLD = 0.05

In [26]:
baseline_shapiro_result = shapiro(baseline_paired_df['num_stories'])
if baseline_shapiro_result.pvalue < THRESHOLD:
    print('Not normal')
else:
    print('Normal')

Not normal


## Compressed

In [27]:
compressed_shapiro_result = shapiro(compressed_paired_df['num_stories'])
if compressed_shapiro_result.pvalue < THRESHOLD:
    print('Not normal')
else:
    print('Normal')

Not normal


Since both settings are not normal, non-parametric tests will be used.

# Mann-Whitney U test 

In [29]:
# Perform Mann-Whitney U test
wilcoxon_result = wilcoxon(baseline_paired_df['num_stories'], compressed_paired_df['num_stories'])

In [30]:
wilcoxon_result

WilcoxonResult(statistic=14.0, pvalue=0.049420750659580896)

# Cohen's D

In [31]:
# Perform Cohen's D
effect_size = pg.compute_effsize(baseline_paired_df['num_stories'], compressed_paired_df['num_stories'], eftype='cohen')

In [32]:
effect_size

-0.1558084982618848

In [ ]:
"Our analysis revealed a small effect size (Cohen’s d = 0.049420750659580896), indicating a substantial difference between the two groups."